In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("krkopt.data", header=None)

In [3]:
data.columns = ["wkc", "wkr", "wrc", "wrr", "bkc", "bkr", "opt rank" ]

In [4]:
data

,wkc,wkr,wrc,wrr,bkc,bkr,opt rank
0,a,1,b,3,c,2,draw
1,a,1,c,1,c,2,draw
2,a,1,c,1,d,1,draw
3,a,1,c,1,d,2,draw
4,a,1,c,2,c,1,draw
...,...,...,...,...,...,...,...
28051,b,1,g,7,e,5,sixteen
28052,b,1,g,7,e,6,sixteen
28053,b,1,g,7,e,7,sixteen
28054,b,1,g,7,f,5,sixteen


In [5]:
X = data.iloc[:, 0:6]
y = data['opt rank']

In [6]:
X

,wkc,wkr,wrc,wrr,bkc,bkr
0,a,1,b,3,c,2
1,a,1,c,1,c,2
2,a,1,c,1,d,1
3,a,1,c,1,d,2
4,a,1,c,2,c,1
...,...,...,...,...,...,...
28051,b,1,g,7,e,5
28052,b,1,g,7,e,6
28053,b,1,g,7,e,7
28054,b,1,g,7,f,5


In [7]:
X["wkc"]=X["wkc"].astype('category')
X["wrc"]=X["wrc"].astype('category')
X["bkc"]=X["bkc"].astype('category')

In [8]:
X["wkc"]=X["wkc"].cat.codes
X["wrc"]=X["wrc"].cat.codes
X["bkc"]=X["bkc"].cat.codes


In [9]:
X

,wkc,wkr,wrc,wrr,bkc,bkr
0,0,1,1,3,2,2
1,0,1,2,1,2,2
2,0,1,2,1,3,1
3,0,1,2,1,3,2
4,0,1,2,2,2,1
...,...,...,...,...,...,...
28051,1,1,6,7,4,5
28052,1,1,6,7,4,6
28053,1,1,6,7,4,7
28054,1,1,6,7,5,5


In [10]:
y = y.astype('category')
y = y.cat.codes

In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split

In [12]:
import keras
from keras.utils import to_categorical


In [61]:

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y, test_size=0.2,
                                                   random_state = 1)

from imblearn.over_sampling import SMOTE
oversample = SMOTE()


X_smote, y_smote = oversample.fit_resample(X, y)

X_train_smote, X_test_smote, y_train_smote, y_test_smote = train_test_split(X_smote,
                                                   y_smote, test_size=0.2,
                                                   random_state = 1)



y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)
y_train_smote = to_categorical(y_train_smote)
y_test_smote = to_categorical(y_test_smote)

In [52]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

scaler = StandardScaler().fit(X_train_smote)
X_train_smote = scaler.transform(X_train_smote)
X_test_smote = scaler.transform(X_test_smote)

In [95]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

checkpoint_filepath = '/tmp/checkpoint'


In [96]:
def make_my_model_multi( units_per_layer, input_s, output_s, activation_='relu'):
    model = Sequential()
    depth = len(units_per_layer)
    model.add(Dense(units_per_layer[0], activation=activation_, input_shape=(input_s,)))
    for i in range(1, depth):
        model.add(Dense(units_per_layer[i], activation=activation_))
    model.add(Dense(output_s, activation = 'softmax'))   
    
    return model

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, verbose=True)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=True)


#En este caso, usamos categorical_crossentropy como función de coste y 
#permitimos elegir otro batch para poder hacer uso de 
def compile_fit_multiclass(model, X_train, X_test, y_train, batch, epochs, verbose=0):
    model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch, verbose=verbose, validation_split=0.1, callbacks = [early_stopping, model_checkpoint])
    model.load_weights(checkpoint_filepath)
    predictions = model.predict(X_test)
    return predictions

def compute_metrics_multiclass(y_test, y_pred):
    results=[]
    results.append(precision_score(y_test, np.round(y_pred), average="micro"))
    results.append(recall_score(y_test, np.round(y_pred), average="micro"))
    results.append(f1_score(y_test, np.round(y_pred), average="micro"))
    results.append(cohen_kappa_score(y_test, np.round(y_pred)))
    return results

In [17]:
y.shape

(28056,)

In [18]:
seed = 1
np.random.seed(seed)
model = make_my_model_multi([100, 100, 100, 100], 6, 18, activation_='relu' )

In [19]:
y_pred = compile_fit_multiclass(model, X_train, X_test, y_train, 256, 200, verbose=1)

Epoch 1/200
88/88 [==============================] - 1s 2ms/step - loss: 2.4724 - accuracy: 0.1922
Epoch 2/200
88/88 [==============================] - 0s 2ms/step - loss: 1.6377 - accuracy: 0.3953
Epoch 3/200
88/88 [==============================] - 0s 1ms/step - loss: 1.3997 - accuracy: 0.4691
Epoch 4/200
88/88 [==============================] - 0s 1ms/step - loss: 1.2755 - accuracy: 0.5050
Epoch 5/200
88/88 [==============================] - 0s 1ms/step - loss: 1.2194 - accuracy: 0.5136
Epoch 6/200
88/88 [==============================] - 0s 1ms/step - loss: 1.1709 - accuracy: 0.5332
Epoch 7/200
88/88 [==============================] - 0s 1ms/step - loss: 1.1219 - accuracy: 0.5534
Epoch 8/200
88/88 [==============================] - 0s 1ms/step - loss: 1.0856 - accuracy: 0.5659
Epoch 9/200
88/88 [==============================] - 0s 1ms/step - loss: 1.0505 - accuracy: 0.5760
Epoch 10/200
88/88 [==============================] - 0s 1ms/step - loss: 1.0148 - accuracy: 0.5871
Epoch 11/

In [20]:
from sklearn.metrics import confusion_matrix, precision_score, \
f1_score, cohen_kappa_score, recall_score

confusion = confusion_matrix(np.argmax(y_pred, axis = 1), np.argmax(y_test, axis = 1))
print(confusion)

[[580   0   2   1   0   0   0   1   0   1   1   0   0   0   0   0   0   0]
 [  1 220   0   0   1   0   0  35   0  35   2   0   4   0   0   0   0   0]
 [  0   0 468   0   0   0   0  11   0   0   0   0  62   5   0  41   0   0]
 [  0   0   0 378   0   0  30   0   0   0   0   9   0   0   0   0   0   0]
 [  0   0   0   0  62   5   0   0   0   0   7   0   0   0   0   0   0   0]
 [  0   0   0   0   1  36   0   0   0   1   0   0   0   0   4   0   0   0]
 [  0   0   0  55   0   0 809   0   0   0   0   0   0  82   0   6   0   0]
 [  0  30   5   0   0   0   0 255   0   8   0   0  35   0   0   0   0   1]
 [  1   0   0   0   0   0   0   0  11   1   0   0   0   0   0   0   0   2]
 [  0  17   0   0   1   0   0   3   0  92   8   0   1   0   0   0   1   0]
 [  0   2   0   0  15   2   0   3   0  23 103   0   0   0   0   0   0   0]
 [  0   0   0   5   0   0   0   0   0   0   0  60   0   0   0   0   0   0]
 [  1   2  29   0   0   0   0  20   0   3   0   0 273   0   0   7   0   0]
 [  1   0   6   0   0   0

In [21]:
compute_metrics_multiclass(np.argmax(y_pred, axis = 1), np.argmax(y_test, axis = 1))

[0.8373129009265858,
 0.8373129009265858,
 0.8373129009265857,
 0.8180928161347001]

In [24]:
import imblearn
from collections import Counter

In [28]:
counter = Counter(y)
for k,v in counter.items():
	per = v / len(y) * 100
	print('Class=%d, n=%d (%.3f%%)' % (k, v, per))

Class=0, n=2796 (9.966%)
Class=17, n=27 (0.096%)
Class=8, n=78 (0.278%)
Class=16, n=246 (0.877%)
Class=14, n=81 (0.289%)
Class=5, n=198 (0.706%)
Class=4, n=471 (1.679%)
Class=10, n=592 (2.110%)
Class=9, n=683 (2.434%)
Class=1, n=1433 (5.108%)
Class=7, n=1712 (6.102%)
Class=12, n=1985 (7.075%)
Class=2, n=2854 (10.173%)
Class=15, n=3597 (12.821%)
Class=13, n=4194 (14.949%)
Class=6, n=4553 (16.228%)
Class=3, n=2166 (7.720%)
Class=11, n=390 (1.390%)


In [31]:
X_train.shape, np.argmax(y_train, axis=1).shape

((22444, 6), (22444,))

In [32]:
X_train_smote.shape, y_train_smote.shape

((65592, 6), (65592, 18))

In [97]:
np.random.seed(seed)
model = make_my_model_multi([100, 100, 100, 100], 6, 18, activation_='relu' )
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 100)               700       
_________________________________________________________________
dense_11 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_12 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_13 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_14 (Dense)             (None, 18)                1818      
Total params: 32,818
Trainable params: 32,818
Non-trainable params: 0
_________________________________________________________________


In [98]:
X_train.shape, X_train_smote.shape, y_train.shape, y_train_smote.shape, X_test.shape, y_test.shape

((22444, 6), (65563, 6), (22444, 18), (65563, 18), (5612, 6), (5612, 18))

In [99]:
help(compile_fit_multiclass)

Help on function compile_fit_multiclass in module __main__:

compile_fit_multiclass(model, X_train, X_test, y_train, batch, epochs, verbose=0)
    #En este caso, usamos categorical_crossentropy como función de coste y 
    #permitimos elegir otro batch para poder hacer uso de



In [129]:
tf.random.set_seed(
    seed
)
y_pred_smote = compile_fit_multiclass(model, X_train_smote, X_test, y_train_smote, 256, 100, verbose=1)

Epoch 1/100
231/231 [==============================] - 1s 2ms/step - loss: 0.1681 - accuracy: 0.9377 - val_loss: 0.2655 - val_accuracy: 0.9166

Epoch 00001: val_loss did not improve from 0.26111
Epoch 2/100
231/231 [==============================] - 0s 2ms/step - loss: 0.1465 - accuracy: 0.9483 - val_loss: 0.2820 - val_accuracy: 0.9059

Epoch 00002: val_loss did not improve from 0.26111
Epoch 3/100
231/231 [==============================] - 0s 2ms/step - loss: 0.1529 - accuracy: 0.9446 - val_loss: 0.2888 - val_accuracy: 0.9016

Epoch 00003: val_loss did not improve from 0.26111
Epoch 4/100
231/231 [==============================] - 0s 2ms/step - loss: 0.1591 - accuracy: 0.9417 - val_loss: 0.2795 - val_accuracy: 0.9123

Epoch 00004: val_loss did not improve from 0.26111
Epoch 5/100
231/231 [==============================] - 0s 2ms/step - loss: 0.1533 - accuracy: 0.9452 - val_loss: 0.2849 - val_accuracy: 0.9097

Epoch 00005: val_loss did not improve from 0.26111
Epoch 6/100
231/231 [====

In [130]:
model.fit(X_test_smote, y_test_smote)

513/513 [==============================] - 1s 1ms/step - loss: 0.7905 - accuracy: 0.7641


In [131]:
y_pred_smote.shape, y_test.shape

((5612, 18), (5612, 18))

In [132]:
confusion = confusion_matrix(np.argmax(y_pred_smote, axis = 1), np.argmax(y_test, axis = 1))
print(confusion)

[[569   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0]
 [  1 244   0   0   0   0   0   9   0   5   2   0   1   0   0   0   0   0]
 [  0   0 529   0   0   0   1   2   0   0   0   0  26   2   0  67   0   0]
 [  0   0   0 424   0   0  55   0   0   0   0   1   0   0   0   0   0   0]
 [  3   0   0   0  77   1   0   0   0   0   1   0   0   0   0   0   0   0]
 [  2   0   0   0   0  42   0   0   0   0   1   0   0   0   1   0   0   0]
 [  0   0   0  12   0   0 806   0   0   0   0   0   0  57   0   1   0   0]
 [  0  16   3   0   2   0   0 307   0   0   2   0  12   0   0   1   0   0]
 [  0   0   0   0   0   0   0   0  11   0   0   0   0   0   0   0   0   0]
 [  7  10   1   0   0   0   0   1   0 155   3   0   1   0   0   0   0   0]
 [  1   0   0   0   1   0   0   1   0   4 112   0   0   0   0   0   0   0]
 [  0   0   0   3   0   0   0   0   0   0   0  68   0   0   0   0   0   0]
 [  0   1  14   0   0   0   0  11   0   0   0   0 343   1   0   4   0   0]
 [  0   0   0   0   0   0

In [133]:
compute_metrics_multiclass(np.argmax(y_pred_smote, axis = 1), np.argmax(y_test, axis = 1))

[0.9068068424803991,
 0.9068068424803991,
 0.9068068424803991,
 0.8960528113126178]

In [125]:
np.argmax(y_pred_smote, axis=1)

array([0, 0, 0, ..., 6, 2, 2])

In [134]:
#model.save("de_momento_mejor.h5")